# 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 02. Importing data

In [2]:
# Create path
path = r'/Users/chadstacey/04-01-2021_InstaCart_Basket_Analysis'

In [4]:
# Import customer dataset
df_cust = pd.read_csv(os.path.join(path, '02_Data', 'Original_Data', 'customers.csv'))

In [5]:
# Check output 
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
# Check shape 
df_cust.shape

(206209, 10)

# 03. Data wrangling

In [7]:
# Renaming column 'Surnam'
df_cust.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [8]:
# Renaming column 'n_dependants'
df_cust.rename(columns = {'n_dependants' : 'dependants'}, inplace = True)

In [9]:
# Check output 
df_cust.head()

,user_id,First Name,Surname,Gender,STATE,Age,date_joined,dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


I've decided not to drop any columns as all could be useful in final analysis 

In [13]:
# Check data types
df_cust.dtypes

user_id         int64
First Name     object
Surname        object
Gender         object
STATE          object
Age             int64
date_joined    object
dependants      int64
fam_status     object
income          int64
dtype: object

# 04. Data quality and consistency checks 

In [14]:
# Describe dataframe
df_cust.describe()

,user_id,Age,dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


All these values look correct so I'm happy to move on to the next stage of cleaning. 

In [16]:
# check for mixed type data 
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


In [17]:
# Change dtype of First Name to string
df_cust['First Name'] = df_cust['First Name'].astype('str')

In [19]:
# Find missing values 
df_cust.isnull().sum()

user_id        0
First Name     0
Surname        0
Gender         0
STATE          0
Age            0
date_joined    0
dependants     0
fam_status     0
income         0
dtype: int64

No missing values :-)

In [20]:
# Look for duplicates
df_cust_dups = df_cust[df_cust.duplicated()]

In [21]:
df_cust_dups

,user_id,First Name,Surname,Gender,STATE,Age,date_joined,dependants,fam_status,income


No duplicates either :-)

# 05. Combine customer data with up-to-date InstaCart data

In [23]:
# Import up-to-date InstaCart data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02_Data', 'Prepared_Data', 'ords_prods_with_flags.pkl'))

In [24]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_purchase_price,spending_flag,median_days_since_prior_order,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [26]:
# Merge the two dataframes
ords_prods_cust = ords_prods_merge.merge(df_cust, on = 'user_id')

In [27]:
# Check shape 
ords_prods_cust.shape

(32404859, 32)

In [28]:
# Check output 
ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,First Name,Surname,Gender,STATE,Age,date_joined,dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [30]:
ords_prods_cust.shape

(32404859, 32)

# 06. Export merged dataframe

In [29]:
# Export data to pkl
ords_prods_cust.to_pickle(os.path.join(path, '02_Data','Prepared_Data', 'orders_products_customers_merged.pkl'))